In [ ]:

import numpy as np
import pandas as pd
import re
import string
import nltk
nltk.download("all")
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# In read_csv encoding cmd is fornon-ASCII characters
df =pd.read_csv("sms_nb.csv")
df.head()
df.shape

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    |   Package bcp47 is already up-to-dat

True

,Label,Text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


(5572, 2)

In [ ]:
#Convert the lable into numerics
df.Label.replace('spam', 1, inplace=True)
df.Label.replace('ham', 0, inplace=True)
df.head()

,Label,Text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
df.dtypes

Label     int64
Text     object
dtype: object

In [ ]:


# Your DataFrame 'df' should already be defined here

# Now let's peform text cleaning on SMS messages
def clean_text(text):
# Convert the text to lowercase
  text = text.lower()
# Remove special characters and digits
  text = re.sub (r'[^a-z\s]', ' ', text)
# Remove URLs- r'http\S+|www.\S+' matches any website with http or www
  text = re.sub (r'http\S+|www.\S+', ' ', text)
# Remove Punctuations- r'[^\w\s]' matches any character that is not a word character
  text = re.sub (r'[^\w\s]', ' ', text)
# Remove Numbers- r'\d+' matches any digits (0-9)
  text = re.sub(r'\d+', ' ', text)
#Remove extra white spaces- r'\s+' matches any whitespaces, tabs, lines
  text = re.sub(r'\s+', ' ', text).strip()
# Remove @ mentions- It removes '@' followed by one or more alphanumeric characters
  text = re.sub(r'@\w+', ' ', text)
#Remove hashtags- It removes '#' followed by one or more alphanumeric characters
  text = re.sub (r'#[A-Za-z0-9]+', ' ', text)
  #Remove special characters- Removes anything which is not alphanumeric characters
  text = re.sub(r'[^\w\s]', ' ', text)
#Tokenize the text
  tokens = nltk.word_tokenize(text)
# Remove stop words
  stop_words = set (stopwords.words ('english'))
  tokens = [x for x in tokens if x not in stop_words]
# Lemmatize the text
  lemmatizer = WordNetLemmatizer()
  tokens = [lemmatizer.lemmatize(x) for x in tokens]
# Join the tokens back into a string
  text = ' '.join(tokens)
  return text

# Apply the clean_text function to the 'Text' column and create a new 'Text_Clean' column
df['Text_Clean'] = df['Text'].apply(clean_text)
df.head()

,Label,Text,Text_Clean
0,0,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n great ...
1,0,Ok lar... Joking wif u oni...,ok lar joking wif u oni
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,free entry wkly comp win fa cup final tkts st ...
3,0,U dun say so early hor... U c already then say...,u dun say early hor u c already say
4,0,"Nah I don't think he goes to usf, he lives aro...",nah think go usf life around though


In [ ]:
# Export csv file to visualize in spreadsheet
df.to_csv('cleaned_data.csv')

In [ ]:
# Now let's begin with NB modeling by creatings features (x) and label (y)
x = df.Text_Clean
y= df.Label
x.head()
y.head()
x.shape
print()
y.shape
df.info()

0    go jurong point crazy available bugis n great ...
1                              ok lar joking wif u oni
2    free entry wkly comp win fa cup final tkts st ...
3                  u dun say early hor u c already say
4                  nah think go usf life around though
Name: Text_Clean, dtype: object

0    0
1    0
2    1
3    0
4    0
Name: Label, dtype: int64

(5572,)

(5572,)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Label       5572 non-null   int64 
 1   Text        5572 non-null   object
 2   Text_Clean  5572 non-null   object
dtypes: int64(1), object(2)
memory usage: 130.7+ KB


In [ ]:

# Assuming you have a DataFrame df

# Convert 'Text' column to string
df['Text'] = df['Text'].astype(str)

# Convert 'Text_Clean' column to string
df['Text_Clean'] = df['Text_Clean'].astype(str)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Label       5572 non-null   int64 
 1   Text        5572 non-null   object
 2   Text_Clean  5572 non-null   object
dtypes: int64(1), object(2)
memory usage: 130.7+ KB


In [ ]:
vectorizer = TfidfVectorizer()
x_vect = vectorizer.fit_transform(x)
print(x_vect.shape) # of features are 7881 below
print(type(x_vect))
print(vectorizer.get_feature_names_out() [0:10])# print the first 10 words/Features print (x_vect) # print each feature and the TFIDF score
print (x_vect)
print(vectorizer.get_feature_names_out()[7000])


(5572, 7067)
<class 'scipy.sparse._csr.csr_matrix'>
['aa' 'aah' 'aaniye' 'aaooooright' 'aathi' 'ab' 'abbey' 'abdomen' 'abeg'
 'abel']
  (0, 6711)	0.19639203497212881
  (0, 210)	0.3514915155008546
  (0, 2494)	0.16277930333789384
  (0, 1080)	0.296941452667904
  (0, 818)	0.33553664491622515
  (0, 3295)	0.28441044083584954
  (0, 6905)	0.23562915911677088
  (0, 2530)	0.1941641068265434
  (0, 820)	0.296941452667904
  (0, 428)	0.2629041905332462
  (0, 1327)	0.2722059831813756
  (0, 4574)	0.24000583809207612
  (0, 3182)	0.3514915155008546
  (0, 2459)	0.15485344845771118
  (1, 4234)	0.5466243141314314
  (1, 6818)	0.43162957585464123
  (1, 3152)	0.5236804332035243
  (1, 3324)	0.4083258549263009
  (1, 4208)	0.2718944069420321
  (2, 291)	0.18816818026525658
  (2, 4886)	0.1800420188056223
  (2, 6415)	0.1380224544873437
  (2, 5793)	0.21922471060483015
  (2, 4835)	0.18636175467482702
  (2, 4936)	0.18636175467482702
  :	:
  (5567, 4726)	0.24554519598456814
  (5567, 4439)	0.2646132036333892
  (5567, 48

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_vect, y)
x_train.shape
x_test.shape
y_train.shape
y_test.shape

(4179, 7067)

(1393, 7067)

(4179,)

(1393,)

In [ ]:
# Build the NB model and make predictions on teh test data to get classification report
from sklearn.metrics import classification_report as cr
clf = MultinomialNB()
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print (cr(y_test, y_pred))


MultinomialNB()

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      1174
           1       1.00      0.76      0.86       219

    accuracy                           0.96      1393
   macro avg       0.98      0.88      0.92      1393
weighted avg       0.96      0.96      0.96      1393



In [ ]:
# Make predictions for the entire data and append into the original data
df['Predicted'] = clf.predict(x_vect)
df.sample(10)

,Label,Text,Text_Clean,Predicted
5428,0,You didnt complete your gist oh.,didnt complete gist oh,0
794,0,Omg I want to scream. I weighed myself and I l...,omg want scream weighed lost weight woohoo,0
2938,0,Lol yep did that yesterday. Already got my fir...,lol yep yesterday already got fireplace anothe...,0
1534,0,Horrible bf... I now v hungry...,horrible bf v hungry,0
758,0,"Faith makes things possible,Hope makes things ...",faith make thing possible hope make thing work...,0
2935,0,Yup n her fren lor. I'm meeting my fren at 730.,yup n fren lor meeting fren,0
404,0,None of that's happening til you get here though,none happening til get though,0
4001,1,This is the 2nd time we have tried to contact ...,nd time tried contact u u prize claim easy cal...,1
483,0,Watching tv lor...,watching tv lor,0
1136,0,K do I need a login or anything,k need login anything,0


In [ ]:
# Calculate the probability of the word 'free' given 'spam'
word_index = vectorizer.vocabulary_['free']
probability_free_given_spam = clf.feature_log_prob_[1][word_index]  # Log probability

# Calculate the probability of the word 'free' given 'ham'
probability_free_given_ham = clf.feature_log_prob_[0][word_index]  # Log probability

# Convert log probabilities to actual probabilities
probability_free_given_spam = 10 ** probability_free_given_spam
probability_free_given_ham = 10 ** probability_free_given_ham

print(f"Probability of 'free' given 'spam': {probability_free_given_spam:.6f}")
print(f"Probability of 'free' given 'ham': {probability_free_given_ham:.6f}")


Probability of 'free' given 'spam': 0.000002
Probability of 'free' given 'ham': 0.000000
